In [1]:
import joblib
import os
import numpy as np
import pandas as pd
import datetime
import Funtions as fn
from sklearn.linear_model import LinearRegression

In [2]:
loaded_data = joblib.load('data/modal_biaya_harian.pkl')

In [3]:
loaded_data

,tgl_transaksi,total_modal_biaya
0,2008-01-01,162955350
1,2008-01-02,173626100
2,2008-01-03,156584600
3,2008-01-04,175112000
4,2008-01-05,171359210
...,...,...
360,2008-12-26,173844260
361,2008-12-27,176955700
362,2008-12-28,179438340
363,2008-12-29,185363820


In [4]:
PREDICTION_DAYS = 7
SEQUENCE_LENGTH = 30

In [5]:
scaler, data_scaled, X_train, y_train, X_val, y_val, train_dates, val_dates = fn.prepare_data(loaded_data, SEQUENCE_LENGTH)

In [6]:
joblib.dump(scaler, 'data/scaler.pkl')

['data/scaler.pkl']

In [7]:
dl_models = fn.create_ensemble_models((X_train.shape[1], 1))

In [8]:
trained_dl_models, histories = fn.train_ensemble_models(dl_models, X_train, y_train, X_val, y_val, 150)

Training model 1/3: lstm_model
Epoch 1/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 7s 101ms/step - loss: 0.1976 - val_loss: 0.0792
Epoch 2/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0491 - val_loss: 0.0341
Epoch 3/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0414 - val_loss: 0.0292
Epoch 4/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 23ms/step - loss: 0.0270 - val_loss: 0.0349
Epoch 5/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0325 - val_loss: 0.0297
Epoch 6/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0280 - val_loss: 0.0282
Epoch 7/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0297 - val_loss: 0.0307
Epoch 8/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0320 - val_loss: 0.0310
Epoch 9/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 22ms/step - loss: 0.0285 - val_loss: 0.0293
Epoch 10/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step - loss: 0.0320 - val_loss: 0.0291
Epoch 11/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 25ms/step - loss: 0.0290 - val_loss: 0.0285
Epoch 12/150
9/9 ━━━━━━━━━━━━━━━━━━━━ 0s 2

In [9]:
individual_predictions_train, individual_predictions_val = fn.individual_predictions(trained_dl_models, X_train, X_val)

9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 52ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 1s 72ms/step
9/9 ━━━━━━━━━━━━━━━━━━━━ 2s 105ms/step
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 7ms/step 
3/3 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step 


In [10]:
train_preds = list(individual_predictions_train.values())
val_preds = list(individual_predictions_val.values())

X_stack_train = np.stack(train_preds, axis=1)
X_stack_val = np.stack(val_preds, axis=1)

In [11]:
# Model meta-ensemble
meta_model = LinearRegression()

meta_model.fit(X_stack_train, y_train)

LinearRegression()

In [12]:
# prediksi menggunakan meta model
ensemble_preds_hist_train = meta_model.predict(X_stack_train)
ensemble_preds_hist_val = meta_model.predict(X_stack_val)

In [13]:
last_date = loaded_data['tgl_transaksi'].iloc[-1]
if isinstance(last_date, pd.Timestamp):
    last_date = last_date.date()

future_dates = [last_date + datetime.timedelta(days=i+1) for i in range(PREDICTION_DAYS)]

last_sequence = data_scaled[-SEQUENCE_LENGTH:]

In [14]:
future_predictions = fn.predict_ensemble(trained_dl_models, scaler, last_sequence, meta_model)

In [15]:
# Menampilkan hasil prediksi masa depan dalam bentuk DataFrame
df_future_predictions = pd.DataFrame({
    'Tanggal': future_dates,
    'Prediksi Modal': future_predictions.flatten()
})


In [16]:
df_future_predictions.set_index('Tanggal', inplace=True)
df_future_predictions

,Prediksi Modal
Tanggal,
2008-12-31,1.721510e+08
2009-01-01,1.727039e+08
2009-01-02,1.733460e+08
2009-01-03,1.742249e+08
2009-01-04,1.742232e+08
2009-01-05,1.751882e+08
2009-01-06,1.758161e+08


In [17]:
fn.plot_ensemble_predictions(
    loaded_data,
    train_dates,
    val_dates,
    scaler,
    ensemble_preds_hist_train,
    ensemble_preds_hist_val,
    df_future_predictions
)


In [18]:
fn.plot_future_predictions(future_dates, future_predictions)

In [19]:
mape, mae, mse, rmse = fn.evaluate_model(y_val, ensemble_preds_hist_val)

Evaluasi hasil prediksi pada Data Validasi:
MAE  : 0.1279
MAPE : 27.9088%
MSE  : 0.0287
RMSE : 0.1693


In [20]:
save_dir = "model/peramalan"

for i, model in enumerate(trained_dl_models):
    model_name = model.name
    model_path = os.path.join(save_dir, f"{i+1}{model_name}.h5")
    model.save(model_path)
    print(f"Model {i+1} disimpan ke: {model_path}")


Model 1 disimpan ke: model/peramalan\1lstm_model.h5
Model 2 disimpan ke: model/peramalan\2gru_model.h5
Model 3 disimpan ke: model/peramalan\3bilstm_model.h5


In [21]:
joblib.dump(meta_model, 'model/peramalan/meta_model.pkl')

['model/peramalan/meta_model.pkl']